In [42]:
import os, sys
cwd = os.getcwd()
project_path = cwd[:cwd.find('pygents')+7]
if project_path not in sys.path: sys.path.append(project_path)
os.chdir(project_path) 

#https://stackoverflow.com/questions/41827983/right-way-to-calculate-the-cosine-similarity-of-two-word-frequency-dictionaries
#https://realpython.com/python-counter/

#from scipy.spatial.distance import cosine
#from sklearn.metrics.pairwise import cosine_similarity

import pandas as pd

#force reimport
if 'pygents.util' in sys.modules:
    del sys.modules['pygents.util']
if 'pygents.text' in sys.modules:
    del sys.modules['pygents.text']
if 'pygents.plot' in sys.modules:
    del sys.modules['pygents.plot']
if 'pygents.token' in sys.modules:
    del sys.modules['pygents.token']

from pygents.token import *
from pygents.text import *
from pygents.util import *
from pygents.plot import plot_bars, plot_dict, matrix_plot


In [59]:
import math

def cosine_dic_parts(dic1,dic2):
    numerator = 0
    dena = 0
    for key1,val1 in dic1.items():
        numerator += val1*dic2.get(key1,0.0)
        dena += val1*val1
    denb = 0
    for val2 in dic2.values():
        denb += val2*val2
    return numerator, dena, denb

def cosine_dic(dic1,dic2):
    numerator, dena, denb = cosine_dic_parts(dic1,dic2)
    return numerator/math.sqrt(dena*denb)

x0 = {'c':3,'a':1,'b':2}
x1 = {'a':1,'b':2,'c':3}
x2 = {'a':1,'b':2,'c':0}
x3 = {'a':1,'b':2,'d':3}
x4 = {'a':1,'e':2,'d':3}
x5 = {'a':999,'e':2,'d':3}
x6 = {'a':0.1,'e':2,'d':3}
x7 = {'f':1,'e':2,'d':3}
assert str(cosine_dic(x0,x1)) == "1.0"
assert str(cosine_dic(x0,x2)) == "0.5976143046671968"
assert str(cosine_dic(x1,x2)) == "0.5976143046671968"
assert str(cosine_dic(x1,x3)) == "0.35714285714285715"
assert str(cosine_dic(x1,x4)) == "0.07142857142857142"
assert str(cosine_dic(x1,x5)) == "0.26725950125174264"
assert str(cosine_dic(x1,x6)) == "0.007409643851431125"
assert str(cosine_dic(x1,x7)) == "0.0"

# computes cosine distance based on 2 dicts corresponding to 
# two vectors in complementary two segments of bi-segment vector space
def cosine_dic2(dica1,dicb1,dica2,dicb2):
    numerator1, dena1, denb1 = cosine_dic_parts(dica1,dicb1)
    numerator2, dena2, denb2 = cosine_dic_parts(dica2,dicb2)
    return (numerator1+numerator2)/math.sqrt((dena1+dena2)*(denb1+denb2))
assert str(cosine_dic2(x0,x1,{},{})) == "1.0"
assert str(cosine_dic2(x0,x2,{},{})) == "0.5976143046671968"
assert str(cosine_dic2(x1,x2,{},{})) == "0.5976143046671968"
assert str(cosine_dic2(x1,x3,{},{})) == "0.35714285714285715"
assert str(cosine_dic2(x1,x4,{},{})) == "0.07142857142857142"
assert str(cosine_dic2(x1,x5,{},{})) == "0.26725950125174264"
assert str(cosine_dic2(x1,x6,{},{})) == "0.007409643851431125"
assert str(cosine_dic2(x1,x7,{},{})) == "0.0"
assert str(cosine_dic2(x0,x1,x0,x1)) == "1.0"
assert str(cosine_dic2(x1,x7,x1,x7)) == "0.0"



In [60]:
print(cosine_dic(x0,x1))
print(cosine_dic(x0,x2))
print(cosine_dic(x1,x2))
print(cosine_dic(x1,x3))
print(cosine_dic(x1,x4))
print(cosine_dic(x1,x5))
print(cosine_dic(x1,x6))
print(cosine_dic(x1,x7))


1.0
0.5976143046671968
0.5976143046671968
0.35714285714285715
0.07142857142857142
0.26725950125174264
0.007409643851431125
0.0


In [61]:
brown_chars = FreedomTokenizer(name='data/models/brown_nolines_chars_7a',max_n=7,mode='chars',debug=False)
model_compress_with_loss(brown_chars.model,0.0001)


In [62]:
print(cosine_dic(brown_chars.model[1]['.'],brown_chars.model[1][',']))
print(cosine_dic(brown_chars.model[2]['.'],brown_chars.model[2][',']))

print(cosine_dic(brown_chars.model[1]['.'],brown_chars.model[1][' ']))
print(cosine_dic(brown_chars.model[2]['.'],brown_chars.model[2][' ']))

print(cosine_dic(brown_chars.model[1]['.'],brown_chars.model[1]['a']))
print(cosine_dic(brown_chars.model[2]['.'],brown_chars.model[2]['a']))

print(cosine_dic(brown_chars.model[1]['o'],brown_chars.model[1]['a']))
print(cosine_dic(brown_chars.model[2]['o'],brown_chars.model[2]['a']))


0.9999222003602027
0.9907565768001892
0.12763480631061797
0.8811401825208185
0.22319366970832127
0.31534753618812983
0.7720141235753476
0.8849807028305376


In [64]:
def compute_similiarities(model,arity=1):
    lst = []
    for a in model[0]:
        if len(a) == arity:
            a1 = model[1][a]
            a2 = model[2][a]
            for b in model[0]:
                if len(b) == arity:
                    b1 = model[1][b]
                    b2 = model[2][b]
                    s = cosine_dic2(a1,b1,a2,b2)
                    lst.append((a,b,s))
    return lst
            